In [1]:
import subprocess, os
import numpy as np

In [2]:
def make_madx_lattice(oq, single=False, num=9):
    currents = np.array([0.05721, 0.09124, 0.14642, 0.23386, 0.36538, 0.5449, 0.751935, 0.9288, 1])
    if single:
        ocps = np.zeros(9)
        ocps[num-1] = oq
        currents_actual = currents*ocps
    else:
        currents_actual = currents*oq
#     else:
#         ocps = np.ones(9)*oq*currents
    with open('include/octupoles.madx','w') as f:
        f.write('! octupoles\n') 
        #f.write('! {}\n'.format(currents))
        #!kG/cm3 -> 1/cm4 -> 1/m4
        f.write('oqK = (1.4*0.5/HR)*100*100*100*100;\n')
        f.write('oqI = {};\n'.format(oq))
        #f.write('koq09 := oqI*oqK;\n')
        for i in range(9):            
            f.write('koq{:02d} := {}*oqK; !{}\n'.format(i+1,currents_actual[i],currents[i]))
            
#     !koq01 := koq09*0.05721;
#     !koq02 := koq09*0.09124;
#     !koq03 := koq09*0.14642;
#     !koq04 := koq09*0.23386;
#     !koq05 := koq09*0.36538;
#     !koq06 := koq09*0.5449;
#     !koq07 := koq09*0.751935;
#     !koq08 := koq09*0.9288;

In [3]:
#currents = [5000, 2000, 1500, 1000, 750, 500, 300, 200, 150, 100, 75, 50, 20, 10, 5, 2, 1, 0.1, 0][::-1]
#currents = [500, 300, 200, 150, 100, 75, 50, 20, 10, 5, 4, 3, 2, 1, 0.5, 0.1, 0][::-1]
#currents = [100, 75, 50, 20, 10, 7.5,  5, 4, 3, 2, 1, 0.8, 0.6, 0.5, 0.2, 0.1, 0][::-1]
currents = [5, 4, 2, 1, 0.5, 0.1, 0][::-1]
name_base = 'v852_{}_all_noE_6_12_1.ltr'
#currents.extend([-v for v in currents[:-1]])
print(currents)
for cr in currents:
    name = '{:05d}'.format(int(cr*10)) if cr >= 0 else 'm{:05d}'.format(int(-cr*10))
    print(name)
    
    #make_madx_lattice(cr, single=True, num=9)
    make_madx_lattice(cr)
    resp = subprocess.check_output(["..\madx-win64-gnu.exe", ".\madx2ltr_complete_6_12_8.madx"])
    if 'MAD-X finished normally' not in resp[-100:].decode():
        print(resp[-100:].decode())
        raise Exception
    
    resp = subprocess.check_output(['C:/Users/boss/Downloads/strawberry-perl-5.28.2.1-64bit-portable/perl/bin/perl.exe', 
                                'scripts/madx2ltr_mod_newdefaults.pl',
                                'out/out.lattice', 'out/esave', 'out/out.strong',
                                 name_base.format(name)])
    if not 'attached:' in resp[-30:].decode():
        print(resp[-100:].decode())
        raise Exception

[0, 0.1, 0.5, 1, 2, 4, 5]
00000
00001
00005
00010
00020
00040
00050
